# [Issue 99](https://github.com/starhawking/python-terrascript/issues/99): Copying a Resource object does not work with dict().copy and copy.deepcopy()

## Problem
> I tried to copy a Resource, so that I dont have to specify the same object over and over again
> What I ran:

```python
self.base_instance = osr.openstack_compute_instance_v2( _name="Instance", 
                                                       name="Instance", 
                                                       count="1", 
                                                       availability_zone=self.provider_az[0], 
                                                       image_name="CentOS 7 (LTS)", 
                                                       flavor_id="2004", 
                                                       key_pair="SchoolDocker", 
                                                       network={"name": "noice"}, 
                                                       user_data="", 
                                                       security_groups=[""] )

instance_2 = copy.deepcopy(self.base_instance)
```

```
Traceback:
Traceback (most recent call last): 
File "./main.py", line 78, in <module> config.add_docker() 
File "./main.py", line 47, in add_docker instance_2 = copy.deepcopy(self.base_instance) 
File "/usr/lib/python3.6/copy.py", line 96, in copy rv = reductor(4) TypeError: 'Attribute' object is not callable
```

> What I also tried:

```python
instance_2 = self.base_instance.copy()
```

> Does also not work

## Analysis
Trying to replicate the problem with other resources, data sources, etc.

In [2]:
import terrascript
import terrascript.provider
import terrascript.resource
import terrascript.data
import copy

### ``terrascript.Terrascript()``

In [3]:
ts = terrascript.Terrascript()
ts

{}

In [4]:
ts2 = copy.copy(ts)
ts2

{}

### ``terrascript.provider.aws()``

In [5]:
provider = terrascript.provider.aws(version='~> 2.0', region='us-east-1')
provider

{'version': '~> 2.0', 'region': 'us-east-1'}

In [6]:
provider2 = copy.copy(provider)
provider2

{'version': '~> 2.0', 'region': 'us-east-1'}

In [7]:
provider3 = copy.copy(provider)
provider3

{'version': '~> 2.0', 'region': 'us-east-1'}

### ``terrascript.resource.aws_vpc()``

In [8]:
resource = terrascript.resource.aws_vpc('example', cidr_block='10.0.0.0/16')
resource

{'cidr_block': '10.0.0.0/16'}

In [10]:
resource2 = copy.copy(resource)
resource2

TypeError: 'Attribute' object is not callable

In [12]:
%%debug
resource3 = copy.copy(resource)
resource3

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(2)<module>()

ipdb> b /home/mjuenemann/.virtualenvs/terrascript/lib64/python3.7/copy.py:95
Breakpoint 1 at /home/mjuenemann/.virtualenvs/terrascript/lib64/python3.7/copy.py:95
ipdb> c
> /home/mjuenemann/.virtualenvs/terrascript/lib64/python3.7/copy.py(95)copy()
     93     else:
     94         reductor = getattr(x, "__reduce_ex__", None)
1--> 95         if reductor:
     96             rv = reductor(4)
     97         else:

ipdb> p x
{'cidr_block': '10.0.0.0/16'}
ipdb> p reductor
<built-in method __reduce_ex__ of aws_vpc object at 0x7f90fce3ac50>
ipdb> q


So it appears that the ``copy.copy()`` function calls ``terrascript.resource.aws.aws_vpc.__reduce_ex__()`` but somehow an instance of ``terrascript.Attribute`` "appears" which is not callable.

``terrascript.Attribute`` is normally meant to be returned in-lieu of non-existent attributes to prevent ``AttributeError`` exceptions. That's one of the tricks of *python-terrascript* to be able to reference them like *Terraform* configurations.

The examples below show this.

In [24]:
# Existing attribute
resource.cidr_block

'10.0.0.0/16'

In [25]:
# Non-existing attribute
resource.does_not_exist, type(resource.does_not_exist)

('aws_vpc.example.does_not_exist', terrascript.Attribute)

In [23]:
# Cannot call `terrascript.Attribute` instances.
resource.does_not_exist()

TypeError: 'Attribute' object is not callable

What I don't understand is why this happens for the ``resource.__reduce_ex__`` attribute which does exists. 

In [22]:
resource.__reduce_ex__, type(resource.__reduce_ex__)

(<function aws_vpc.__reduce_ex__(protocol, /)>, builtin_function_or_method)

According to the [Python documentation of the Pickle module](https://docs.python.org/3/library/pickle.html), the ``__reduce_ex__()`` and ``__reduce__()`` methods are used top serialise a Python object. It's interestingto know that ``copy.copy()`` seems to pickle/unpickle an object.

Let's check why an instance of ``terrascript.Attribute`` is returned instead of the ``__reduce_ex__`` method.

In [13]:
%%debug
resource3 = copy.copy(resource)
resource3

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(2)<module>()

ipdb> b /home/mjuenemann/.virtualenvs/terrascript/lib/python3.7/site-packages/terrascript-0.8.0-py3.7.egg/terrascript/__init__.py:108
Breakpoint 1 at /home/mjuenemann/.virtualenvs/terrascript/lib/python3.7/site-packages/terrascript-0.8.0-py3.7.egg/terrascript/__init__.py:108
ipdb> c
> /home/mjuenemann/.virtualenvs/terrascript/lib/python3.7/site-packages/terrascript-0.8.0-py3.7.egg/terrascript/__init__.py(108)__getattr__()
    106         # which must be formatted differently depending on what is referenced.
    107         #
1-> 108         if attr in self:
    109             return self[attr]
    110         else:

ipdb> p attr
'__getstate__'
ipdb> q


The Python documentation says:
> Classes can further influence how their instances are pickled; if the class defines the method ``__getstate__()``, 
> it is called and the returned object is pickled as the contents for the instance, instead of the contents 
> of the instance’s dictionary. If the ``__getstate__``() method is absent, the instance’s ``__dict__`` is pickled as usual.

In [26]:
resource.__getstate__, type(resource.__getstate__)

('aws_vpc.example.__getstate__', terrascript.Attribute)

So one could create a ``.__getstate__()`` method to prevent ``terrascript.Attribute`` from being returned but what would ``.__getstate__()`` have to return?

### ``terrascript.Attribute``
This class is normally not used directly.

In [16]:
a = terrascript.Attribute()
a

''

In [17]:
copy.copy(a)

TypeError: 'Attribute' object is not callable

``terrascript.Attribute`` is just a sub-class of ``str`` which **can** be copied.

In [20]:
s = 'this is a string'
copy.copy(s)

'this is a string'

In [21]:
resource.keys()

dict_keys(['cidr_block'])